In [0]:
import torch
from torch import nn
import numpy as np
from torch.nn import functional as F

In [0]:
embedding_size = 300
hidden_layers = 1
hidden_size = 64
output_size = 4
max_epochs = 15
hidden_size_linear = 128
lr = 0.5
batch_size = 256
dropout_rate = 0.7

In [0]:
class RCNN(nn.Module):
  def __init__(self, vocab_size, embeddings):
    super(RCNN, self).__init__()

    # embedding layer
    self.embeddings = nn.Embedding(vocab_size,embedding_size)
    self.embeddings.weight = nn.Parameter(embeddings, requires_grad=False)

    #Use Bi-directional LSTM for RCNN instead of Vanilla RCNN
    self.lstm == nn.LSTM(input_size = embedding_size,
                         hidden_size = hidden_size,
                         num_layers = hidden_layers,
                         dropout = dropout_rate,
                         birectional = True)
    
    self.dropout = nn.Droupout(dropout_rate)

    self.W = nn.Linear(
        embedding_size + 2*hidden_size,
        hidden_size_linear
    )

    self.tanh = nn.Tanh()

    self.fc = nn.Linear(
        hidden_size_linear,
        output_size
    )

    self.softmax = nn.Softmax()


  def forward(self,x):
    embedded_sentence = self.embeddings(x)
    lstm_out, (h_n, c_n) = self.lstm(embedded_sentence)
    input_features = torch.cat([lstm_out,embedded_sentence],2).permute(1,0,2)

    linear_out = self.tanh(
        self.W(input_features)
    )
    linear_out = linear_out.permute(0,2,1)

    max_out_features = F.max_pool1d(linear_output,linear_output.shape[2]).squeeze(2)
    max_out_features = self.dropout(max_out_features)

    final_out = self.fc(max_out_features)
    return self.softmax(final_out)


  def train(self, train_iterator, epoch):
    losses = []

    for i , batch in enumerate(train_iterator):
      self.optimizer.zero_grad()
      if torch.cuda.is_available():
        x = batch.text.cuda()
        y = (batch.label - 1).type(torch.cuda.LongTensor)
      else:
        x = batch.text
        y = (batch.label - 1).type(torch.LongTensor)

      y_pred = self.__call__(x)
      loss = self.loss_op(y_pred,y)
      loss.backward()
      losses.append(loss.data.cpu().numpy())
      self.optimizer.step()

      if i % 100 == 0:
        print("Iter: {}".format(i+1))
        avg_train_loss = np.mean(losses)
        print("\tAverage training loss: {:.4f}".format(avg_train_loss))
